In [0]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [10]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [11]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key =  item['key'].lower().strip()
    value =  item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict



In [0]:
df['features_parsed'] = df['features'].map(parse_features)
df['features_parsed'].head().values

In [0]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

In [16]:
def get_name_feat(key):
  return 'feat_' + key


for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[ get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [18]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]



In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  get_name_feat(key)

In [21]:
df['brand'] = df['brand'].map( lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 1002)

In [0]:
feats = ['']

In [23]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.34544173510458, 4.191169845973905)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [33]:
feats = ['brand_cat', 'feat_gender_cat', 'feat_material_cat']
# feats += feats_cat
# feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.0903877290629, 4.0293679353599305)

In [31]:
X = df[ feats ].values
y =  df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2632 ± 0.0101,brand_cat
0.1062 ± 0.0122,feat_material_cat
0.0446 ± 0.0025,feat_gender_cat
0.0210 ± 0.0011,feat_brand_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat
